## Import packages

In [1]:
# import the packages
from lxml import etree
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import requests 
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import csv
import os
import time
import random

In [2]:
pip install fake-useragent

Note: you may need to restart the kernel to use updated packages.


In [3]:
from fake_useragent import UserAgent
ua = UserAgent()

## set up the links and IPs

In [4]:
baseUrl = 'https://www.leleketang.com/zuowen/'

urlArray = [
  #'list30-0-0-1-1.shtml', 
  #'clist0-0-0-1-1.shtml',
  'list20-0-0-1-1.shtml'
  #'list10-0-0-1-1.shtml'
]

# define different user agents information
user_agent_list = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36",
                   "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0",
                   "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Version/13.1.2 Safari/537.36",
                   "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36 Edg/93.0.961.38",
                   "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
                   "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36 OPR/78.0.4093.231",
                   "Mozilla/5.0 (Linux; Android 11; Pixel 3 XL) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Mobile Safari/537.36",
                   "Mozilla/5.0 (Android 11; Mobile; rv:93.0) Gecko/93.0 Firefox/93.0 SamsungBrowser/17.0"
                   "Mozilla/5.0 (Windows Phone 10.0; Android 6.0.1; Microsoft; RM-1152) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Mobile Safari/537.36 Edge/40.15254.547",
                   "Mozilla/5.0 (iPhone; CPU iPhone OS 15_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1",
                   "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; AS; rv:11.0) like Gecko",
                   "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.190 Safari/537.36",
                   "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko",
                   "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Edge/88.0.705.50 Safari/537.36",
                    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_5_2) AppleWebKit/537.36 (KHTML, like Gecko) Version/14.1.2 Safari/537.36",
                   "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.82"
                    ]

# define different ip addresses
ip_list=[                       
        '118.70.144.77:3128',
        '113.200.105.45:8080'
    ]

# from where
referer_list=[
            'https://www.sogou.com/',
            'https://www.baidu.com/'
        ]

# change ip
ip=random.choice(ip_list)
proxy_ip = 'http://' + ip
proxy_ips = 'https://' + ip
proxy = {'https': proxy_ips, 'http': proxy_ip}

urls = []
evaluate = []

## Define the functions

In [6]:
#request html according to url
def query(url):
    html = requests.get(url=url, headers=headers)
    page = html.content.decode('utf-8', 'ignore')
    return BeautifulSoup(page, 'lxml')

In [7]:
def get_evaluate(page):
    # Get title links
    compositions_rating_5 = page.find_all('a', class_ = 'ellipsis item_rating_5')
    compositions_rating_4 = page.find_all('a', class_ = 'ellipsis item_rating_4')
    compositions_rating_3 = page.find_all('a', class_ = 'ellipsis item_rating_3')
    compositions_rating_2 = page.find_all('a', class_ = 'ellipsis item_rating_2')

    compositions = compositions_rating_5 + compositions_rating_4 + compositions_rating_3 + compositions_rating_2

    # Visit all papers in current page and store
    for title in compositions:
        compositionPage = query(baseUrl + title['href'])
        
        # Get the article rating score
        name_orig = compositionPage.find('h2', class_ = 'cp_htitle')
        if(name_orig):
            name = name_orig.get_text()
        else:
            name = "N/A"
        
        # Get the article rating score
        score = compositionPage.find('div', class_ = 'cp_rating rating5')
        if(score):
            rating = "优"
        else:
            score = compositionPage.find('div', class_ = 'cp_rating rating4')
            if(score):
                rating = "优"
            else:
                score = compositionPage.find('div', class_ = 'cp_rating rating2')
                if(score):
                    rating = "中"
                else:
                    score = compositionPage.find('div', class_ = 'cp_rating rating3')
                    if(score):
                        rating = "良"
                    else:
                        rating = "差"
        
          # Get the comment
        for tag in compositionPage.find_all('div', class_ = "cp_comment"):
            comment = tag.find('p').get_text()
            # Get all content paragraphs and combine them
            content_paragraphs = compositionPage.find_all('div', class_='cp_content')
            content_parts = []
            for paragraph in content_paragraphs:
                for t in paragraph.find_all(['p']):
                    content_parts.append(t.get_text())
            content = ' '.join(content_parts)
            # store the tags
            cat_tot = compositionPage.find_all('div', class_='com_tag')
            cat_parts = []
            counter = 0
            for cat in cat_tot:
                for c in cat.find_all(['a']):
                    if (counter != 0):
                        cat_parts.append("/")  # as a splitter
                    cat_parts.append(c.get_text())  # append the content
                    counter += 1  # cnt + 1
            categories = ' '.join(cat_parts)
                
            # Write all information to the CSV file in the same row
            row = [baseUrl + title['href'], name, rating, categories, content, comment]
            if(row == None):
                print("Not reading at all!")
                return False
            csvwrite.writerow(row)

## Set up the file path and get information from web

In [5]:
# Specify the file path
file_path = 'Your/path/goes/here.csv'

# Check if the file already exists
if os.path.isfile(file_path):
    # If the file exists, delete it
    os.remove(file_path)

# Create a new file
with open(file_path, "w", encoding="utf-8") as file:
    csvwrite = csv.writer(file)
    # Add column headers
    csvwrite.writerow(['Composition Link', 'Title', 'Score', 'Category', 'Article Context', 'Comment'])
    #pass  # No need to write anything, just pass

# Read the file
f = open(file_path, mode = "w", encoding = "utf-8", newline = "")  # w: writter mode, encoding: use utf-8 code to read character
csvwrite = csv.writer(f)  # create a csv writter object


# Define the headers; information from inspection of https://www.leleketang.com/ 
headers = {
    'User-Agent' : ua.random, #random.choice(user_agent_list),
    #'Accept-Language':'zh-CN,zh;q=0.9', 
    'cookie' : "gs_browser=pc; gs_browse_mode=desktop; PHPSESSID=dedb3d57ff60b15d26d25b0631e76ac4; Hm_lvt_8296a32ce52c7db4820be505c8738193=1689086848; gs_anony=O%3A4%3A%22User%22%3A33%3A%7Bs%3A2%3A%22id%22%3Bi%3A1172426712%3Bs%3A6%3A%22fromId%22%3Bs%3A5%3A%22anony%22%3Bs%3A5%3A%22snsId%22%3Bi%3A0%3Bs%3A8%3A%22nickname%22%3Bs%3A16%3A%22%E6%B8%B8%E5%AE%A21172426712%22%3Bs%3A6%3A%22gender%22%3Bi%3A0%3Bs%3A6%3A%22avatar%22%3Bs%3A23%3A%22%2Flogin%2Fimage%2Fanony2.png%22%3Bs%3A5%3A%22space%22%3Bs%3A0%3A%22%22%3Bs%3A5%3A%22email%22%3Bs%3A0%3A%22%22%3Bs%3A6%3A%22mobile%22%3Bs%3A0%3A%22%22%3Bs%3A8%3A%22realName%22%3Bs%3A0%3A%22%22%3Bs%3A8%3A%22location%22%3Bs%3A0%3A%22%22%3Bs%3A5%3A%22brief%22%3Bs%3A0%3A%22%22%3Bs%3A9%3A%22signature%22%3Bs%3A0%3A%22%22%3Bs%3A10%3A%22createTime%22%3Bs%3A0%3A%22%22%3Bs%3A4%3A%22fans%22%3Bi%3A0%3Bs%3A7%3A%22follows%22%3Bi%3A0%3Bs%3A8%3A%22articles%22%3Bi%3A0%3Bs%3A4%3A%22role%22%3Bi%3A0%3Bs%3A8%3A%22roleName%22%3Bs%3A0%3A%22%22%3Bs%3A6%3A%22skinId%22%3Bi%3A0%3Bs%3A8%3A%22birthday%22%3Bs%3A0%3A%22%22%3Bs%3A10%3A%22locationId%22%3Bi%3A0%3Bs%3A9%3A%22famillyId%22%3Bi%3A0%3Bs%3A6%3A%22visits%22%3Bi%3A0%3Bs%3A13%3A%22lastVisitTime%22%3BN%3Bs%3A4%3A%22days%22%3Bi%3A0%3Bs%3A8%3A%22lastDays%22%3Bi%3A0%3Bs%3A5%3A%22coins%22%3Bi%3A0%3Bs%3A4%3A%22gems%22%3Bi%3A0%3Bs%3A11%3A%22experiences%22%3Bi%3A0%3Bs%3A19%3A%22current_experiences%22%3Bi%3A0%3Bs%3A5%3A%22level%22%3Bi%3A0%3Bs%3A6%3A%22awards%22%3Bi%3A0%3B%7D; state=https%3A%2F%2Fwww.leleketang.com%2Flogin%2Fverify.php%3Fstate%3D%2Fzuowen%2Flist30-0-0-96-1.shtml; gs_pass=35266162-e4636387f88ee05f6e39f06195741523; Hm_lpvt_8296a32ce52c7db4820be505c8738193=1689125438",
    # Setting jump from where
    'Connection': 'close',
    'referer': random.choice(referer_list),
    "Content-Type": "application/json"
}

In [ ]:
for index, url in enumerate(urlArray):
    homePage = query(baseUrl + url)

  # Get the total page count for four classes each
    for tag in homePage.find_all('div', class_ = 'p_pager_brief'):
        pageCount = int(tag.find('span').get_text().split('/')[1])

        with ThreadPoolExecutor(100) as t:
        # Loop through all the pages on current class
          for tag in range(0, pageCount):
            page = query(baseUrl + url[:11] + str(tag+1) + url[-8:])
            get_evaluate(page)
            cnt = tag
            time.sleep(6)

## Open file to check

In [ ]:
file_dir = "Your/path/goes/here.csv"
import csv
with open(file_dir,'r', encoding = 'UTF-8') as csvfile:
    data_reader = csv.reader(csvfile)

    data_orig = []
    for row in data_reader:
        data_orig.append(row)

In [ ]:
len(data_orig)

In [ ]:
data_orig[1675]